# [Get first row of dataframe in Python Pandas based on criteria](https://stackoverflow.com/questions/40660088/get-first-row-of-dataframe-in-python-pandas-based-on-criteria)

In [33]:
import pandas as pd
df = pd.DataFrame([[1, 2, 1], [1, 3, 2], [4, 6, 3], [4, 3, 4], [5, 4, 5]], columns=['A', 'B', 'C'])
df

,A,B,C
0,1,2,1
1,1,3,2
2,4,6,3
3,4,3,4
4,5,4,5


For this  want to get the first row that fulfil some criteria. Examples:

1. Get first row where A > 3 (returns row 2)
1. Get first row where A > 4 AND B > 3 (returns row 4)
1. Get first row where A > 3 AND (B > 3 OR C > 2) (returns row 2)

[This tutorial](http://pandas.pydata.org/pandas-docs/stable/10min.html)
is a very good one for pandas slicing. Make sure you check it out. Onto some snippets... To slice a dataframe with a condition, you use this format:

```
>>> df[condition]
```

This will return a slice of your dataframe which you can index using `iloc`. Here are your examples:

1. Get first row where A > 3 (returns row 2)

In [34]:
df[df.A > 3].iloc[0]

A    4
B    6
C    3
Name: 2, dtype: int64

If what you actually want is the row number, rather than using `iloc`, it would be `df[df.A > 3].index[0]`.

2. Get first row where A > 4 AND B > 3:

In [36]:
df[(df.A > 4) & (df.B > 3)].iloc[0]

A    5
B    4
C    5
Name: 4, dtype: int64

3. Get first row where A > 3 AND (B > 3 OR C > 2) (returns row 2)

In [37]:
df[(df.A > 3) & ((df.B > 3) | (df.C > 2))].iloc[0]

A    4
B    6
C    3
Name: 2, dtype: int64

Now, with your last case we can write a function that handles the default case of returning the descending-sorted frame:

In [38]:
def series_or_default(X, condition, default_col, ascending=False):
    sliced = X[condition]
    if sliced.shape[0] == 0:
        return X.sort_values(default_col, ascending=ascending).iloc[0]
    return sliced.iloc[0]

series_or_default(df, df.A > 6, 'A')

A    5
B    4
C    5
Name: 4, dtype: int64

# [Interesting Ways to Select Pandas DataFrame Columns](https://towardsdatascience.com/interesting-ways-to-select-pandas-dataframe-columns-b29b82bbfb33)

Casey Whorton - Apr 16·4 min read

<img src="0_MArNnCyyR4B8zsbp.jfif" />
Photo by Cristina Gottardi on Unsplash

Manipulating pandas data frames is a common task during exploratory analysis or preprocessing in a Data Science project. Filtering and sub-setting the data is also common. Over time, I have found myself needing to select columns based on different criteria. I hope readers find this article as a reference.

## Example Data

If you want to use the data I used to test out these methods of selecting columns from a pandas data frame, use the code snippet below to get the wine dataset into your IDE or a notebook.
from sklearn.datasets import load_wine

In [15]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
import re

X = load_wine()
df = pd.DataFrame(X.data, columns = X.feature_names)

df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


Image of Table of Wine Data
Screenshot by Author of Wine Dataset in a Jupyter notebook


Now, depending on what you want to do, check out each one of the code snippets below and try for yourself!

## Selecting columns based on their name

This is the most basic way to select a single column from a dataframe, just put the string name of the column in brackets. Returns a pandas series.


In [16]:
df['hue']


0      1.04
1      1.05
2      1.03
3      0.86
4      1.04
       ... 
173    0.64
174    0.70
175    0.59
176    0.60
177    0.61
Name: hue, Length: 178, dtype: float64

Passing a list in the brackets lets you select multiple columns at the same time.


In [17]:
df[['alcohol','hue']]


,alcohol,hue
0,14.23,1.04
1,13.20,1.05
2,13.16,1.03
3,14.37,0.86
4,13.24,1.04
...,...,...
173,13.71,0.64
174,13.40,0.70
175,13.27,0.59
176,13.17,0.60


## Selecting a subset of columns found in a list

Similar to the previous example, but here you can search over all the columns in the dataframe.


In [29]:
df[df.columns[df.columns.isin(['alcohol','hue','NON-EXISTANT COLUMN'])]]


,alcohol,hue
0,14.23,1.04
1,13.20,1.05
2,13.16,1.03
3,14.37,0.86
4,13.24,1.04
...,...,...
173,13.71,0.64
174,13.40,0.70
175,13.27,0.59
176,13.17,0.60


## Selecting a subset of columns based on difference of columns


Let’s say you know what columns you don’t want in the dataframe. Pass those as a list to the difference method and you’ll get back everything except them.


In [30]:
df[df.columns.difference(['alcohol','hue'])]


,alcalinity_of_ash,ash,color_intensity,flavanoids,magnesium,malic_acid,nonflavanoid_phenols,od280/od315_of_diluted_wines,proanthocyanins,proline,total_phenols
0,15.6,2.43,5.64,3.06,127.0,1.71,0.28,3.92,2.29,1065.0,2.80
1,11.2,2.14,4.38,2.76,100.0,1.78,0.26,3.40,1.28,1050.0,2.65
2,18.6,2.67,5.68,3.24,101.0,2.36,0.30,3.17,2.81,1185.0,2.80
3,16.8,2.50,7.80,3.49,113.0,1.95,0.24,3.45,2.18,1480.0,3.85
4,21.0,2.87,4.32,2.69,118.0,2.59,0.39,2.93,1.82,735.0,2.80
...,...,...,...,...,...,...,...,...,...,...,...
173,20.5,2.45,7.70,0.61,95.0,5.65,0.52,1.74,1.06,740.0,1.68
174,23.0,2.48,7.30,0.75,102.0,3.91,0.43,1.56,1.41,750.0,1.80
175,20.0,2.26,10.20,0.69,120.0,4.28,0.43,1.56,1.35,835.0,1.59
176,20.0,2.37,9.30,0.68,120.0,2.59,0.53,1.62,1.46,840.0,1.65


## Selecting a subset of columns that is not in a list


Return a data frame that has columns that are not in a list that you want to search over.


In [20]:
df[df.columns[~df.columns.isin(['alcohol','hue'])]]


,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,od280/od315_of_diluted_wines,proline
0,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,3.92,1065.0
1,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,3.40,1050.0
2,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,3.17,1185.0
3,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,3.45,1480.0
4,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...
173,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,1.74,740.0
174,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,1.56,750.0
175,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,1.56,835.0
176,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,1.62,840.0


## Selecting columns based on their data type


Data types include ‘float64’ and ‘object’ and are inferred from the columns passed to the dtypes method. By matching on columns that are the same data type, you’ll get a series of True/False. Use the values method to get just the True/False values and not the index.


In [21]:
df.loc[:,(df.dtypes=='float64').values]


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,13.71,5.65,2.45,20.5,95.0,1.68,0.61,0.52,1.06,7.70,0.64,1.74,740.0
174,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0
175,13.27,4.28,2.26,20.0,120.0,1.59,0.69,0.43,1.35,10.20,0.59,1.56,835.0
176,13.17,2.59,2.37,20.0,120.0,1.65,0.68,0.53,1.46,9.30,0.60,1.62,840.0


## Selecting columns based on their column name containing a substring


If you have tons of columns in a data frame and their column names all have a similar substring that you are interested in, you can return the columns who’s names contain a substring. Here we want everything that has the “al” substring in it.


In [22]:
df.loc[:,['al' in i for i in df.columns]]


,alcohol,malic_acid,alcalinity_of_ash,total_phenols
0,14.23,1.71,15.6,2.80
1,13.20,1.78,11.2,2.65
2,13.16,2.36,18.6,2.80
3,14.37,1.95,16.8,3.85
4,13.24,2.59,21.0,2.80
...,...,...,...,...
173,13.71,5.65,20.5,1.68
174,13.40,3.91,23.0,1.80
175,13.27,4.28,20.0,1.59
176,13.17,2.59,20.0,1.65


## Selecting columns based on their column name containing a string wildcard


You could have hundreds of columns, so it might make sense to find columns that match a pattern. Searching for column names that match a wildcard can be done with the “search” function from the re package (see the link in the reference section for more details on using the regular expression package).


In [23]:
df.loc[:,[True if re.search('flava+',column) else False for column in df.columns]]


,flavanoids,nonflavanoid_phenols
0,3.06,0.28
1,2.76,0.26
2,3.24,0.30
3,3.49,0.24
4,2.69,0.39
...,...,...
173,0.61,0.52
174,0.75,0.43
175,0.69,0.43
176,0.68,0.53


## Selecting columns based on how their column name starts


If you want to select columns with names that start with a certain string, you can use the startswith method and pass it in the columns spot for the data frame location.


In [24]:
df.loc[:,df.columns.str.startswith('al')]


,alcohol,alcalinity_of_ash
0,14.23,15.6
1,13.20,11.2
2,13.16,18.6
3,14.37,16.8
4,13.24,21.0
...,...,...
173,13.71,20.5
174,13.40,23.0
175,13.27,20.0
176,13.17,20.0


## Selecting columns based on how their column name ends


Same as the last example, but finds columns with names that end a certain way.


In [25]:
df.loc[:,df.columns.str.endswith('oids')]


,flavanoids
0,3.06
1,2.76
2,3.24
3,3.49
4,2.69
...,...
173,0.61
174,0.75
175,0.69
176,0.68


## Selecting columns if all rows meet a condition


You can pick columns if the rows meet a condition. Here, if all the the values in a column is greater than 14, we return the column from the data frame.


In [26]:
df.loc[:,[(df[col] > 14).all() for col in df.columns]]


,magnesium,proline
0,127.0,1065.0
1,100.0,1050.0
2,101.0,1185.0
3,113.0,1480.0
4,118.0,735.0
...,...,...
173,95.0,740.0
174,102.0,750.0
175,120.0,835.0
176,120.0,840.0


## Selecting columns if any row of a column meets a condition


Here, if any of the the values in a column is greater than 14, we return the column from the data frame.


In [27]:
df.loc[:,[(df[col] > 14).any() for col in df.columns]]


,alcohol,alcalinity_of_ash,magnesium,proline
0,14.23,15.6,127.0,1065.0
1,13.20,11.2,100.0,1050.0
2,13.16,18.6,101.0,1185.0
3,14.37,16.8,113.0,1480.0
4,13.24,21.0,118.0,735.0
...,...,...,...,...
173,13.71,20.5,95.0,740.0
174,13.40,23.0,102.0,750.0
175,13.27,20.0,120.0,835.0
176,13.17,20.0,120.0,840.0


## Selecting columns if the average of rows in a column meet a condition


Here, if the mean of all the values in a column meet a condition, return the column.


In [28]:
df.loc[:,[(df[col].mean() > 7) for col in df.columns]]


,alcohol,alcalinity_of_ash,magnesium,proline
0,14.23,15.6,127.0,1065.0
1,13.20,11.2,100.0,1050.0
2,13.16,18.6,101.0,1185.0
3,14.37,16.8,113.0,1480.0
4,13.24,21.0,118.0,735.0
...,...,...,...,...
173,13.71,20.5,95.0,740.0
174,13.40,23.0,102.0,750.0
175,13.27,20.0,120.0,835.0
176,13.17,20.0,120.0,840.0


Thanks for checking this out and feel free to reference it often.